# Transformer


Arquitectura del transformer:

## Codificador

El codificador se centra en entender la frase de entrada.

2 pasos:

- **Auto atención** (self-attention)
- **Propagación hacia adelante** (feed forward)

Ejem:

'Soy de Málaga'

Como no podemos pasar las palabras al modelo, le pasamos los embedings. Pasamos un embeding por cada palabra al mecanísmo de auto atención.

Ese mecanismo de auto atención, nos devuelve vectores de contexto.

Le pasamos los vectores de contexto al mecanismo de propagación hacia adelante y nos devuelve el restultado.

Así funciona la auto-atencion: https://www.youtube.com/watch?v=8nkOV8UKpNM&list=PLxJ3eugu174Jm7Zj1Gx6Ex8-nGbPHFCEh&index=14


## Decodificador

El decodificador traduce la frase una vez que conoce el significado.

3 pasos:
- **Auto atención** (self-attention)
- **Atención cruzada** (encoder.decoder attention): Le pasamos los vectores de contexto + el resultado del codificador. La query la calculamos con los vectores de contexto y la matriz k y matriz v la calculamos con el resultado del codificador. De esta manera calculamos la atención cruzada entre el codificador y el descodificador. Esto nos devuelve otros vectores de contexto que los pasamos hacia el mecanísmo de propagación hacia adelante.
- **Propagación hacia adelante** (feed forward)

### Pasos

Ayudan a combinar la información nueva con la original y a mantener todo en un rango uniforme para que que el modelo funcione mejor en las próximas etapas.

Frase original: "El gato negro"

Los Transformers primero pasan esta frase por varias capas, incluyendo la capa de autoatención.

**Paso 1: Vectorización**

- "El": [1, 1, 1]
- "gato": [1, 2, 3]
- "negro": [2, 3, 4]

**Paso 2: Linear (Lineal)**

Aplicamos una transformación lineal a estos vectores. Digamos que tenemos una matriz de pesos W que multiplica estos vectores.

W:
(1 0 1
 0 1 0
 1 0 1)
 
Multiplicamos cada vector por W:

- "El": W * [0.3, 0.2, 0.2] = [0.5, 0.2, 0.5]
- "gato": W * [0.3, 0.5, 0.7] = [1.0, 0.5, 1.0]
- "negro": W * [0.5, 0.7, 0.9] = [1.4, 0.7, 1.4]

**Paso 3: Autoatención**

La autoatención procesa cada palabra y decide qué tanto debe prestar atención a las otras palabras en la frase. Por ejemplo, podría decidir que "gato" debería prestar mucha atención a "negro" porque es su descripción.

**Paso 4: Softmax**

Ahora aplicamos Softmax a los resultados para convertirlos en probabilidades.

Calculamos exponentes:

- "El": [exp(0.5), exp(0.2), exp(0.5)]
- "gato": [exp(1.0), exp(0.5), exp(1.0)]
- "negro": [exp(1.4), exp(0.7), exp(1.4)]

Sumamos los exponentes:

- "El": sum(exp(0.5), exp(0.2), exp(0.5))
- "gato": sum(exp(1.0), exp(0.5), exp(1.0))
- "negro": sum(exp(1.4), exp(0.7), exp(1.4))

Calculamos las probabilidades:

- "El": [exp(0.5)/suma, exp(0.2)/suma, exp(0.5)/suma]
- "gato": [exp(1.0)/suma, exp(0.5)/suma, exp(1.0)/suma]
- "negro": [exp(1.4)/suma, exp(0.7)/suma, exp(1.4)/suma]

**Paso 5: Add (Suma)**

Después de la autoatención, obtenemos nuevos vectores que representan las palabras con su nuevo contexto. Pero no queremos perder la información original de las palabras. Así que sumamos (Add) el vector original de cada palabra al nuevo vector obtenido después de la autoatención.

- Vector original "gato": [1, 2, 3]
- Vector después de la autoatención "gato": [2, 3, 4]
- Vector combinado "gato": [1+2, 2+3, 3+4] = [3, 5, 7]

**Paso 6: Norm (Normalización)**

Después de sumar, ajustamos (normalizamos) los valores del vector combinado para que sean más uniformes y estén en un rango manejable. Esto ayuda a que los cálculos en las siguientes capas del Transformer sean más estables y eficientes.

- Vector combinado "gato" (antes de la normalización): [3, 5, 7]
- Vector normalizado "gato": [0.3, 0.5, 0.7] (estos son ejemplos simplificados)


**Paso 7:Feed-Forward Network**

Pasamos el vector normalizado por una red.

El resultado puede ser algo como [2.8, 4.6, 6.1].



---

# Pipeline

In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import pipeline

2024-06-07 09:03:06.868727: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Tarea de clasificación

classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/Users/lunaflorestorres/.pyenv/versions/3.11.6/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
res = classifier('Me encantan las clases de Nechu, explica genial')
print(res)

[{'label': 'POSITIVE', 'score': 0.8972371816635132}]


In [6]:
res = classifier('El profesor es bueno, todo será sencillo')
print(res)

[{'label': 'NEGATIVE', 'score': 0.7578513622283936}]


Esta última frase realmente no es negativa, entonces ¿por qué la está puntuando así? Pues porque aquí:

classifier = pipeline('sentiment-analysis')

No estamos especificando el modelo que queremos usar. El que viene por defento es: 'distilbert-base-uncased-finetuned-sst-2-english'. Y las frases que le hemos pasado no son en inglés.

In [7]:
res = classifier('I love Nechus classes, he explains great.')
print(res)

[{'label': 'POSITIVE', 'score': 0.9997887015342712}]


In [8]:
res = classifier('The teacher is good, everything will be easy')
print(res)

[{'label': 'POSITIVE', 'score': 0.9998722076416016}]


Existe una amplia variedad de 'pipelines': https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline

# Modelo y tokenizador

El modelo lo hemos ejecutado con apenas una línea, pero realmente hay bastantes etapas que ocurren por debajo. En el siguiente código vamos a ver las más importantes.

In [9]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [10]:
# Primero veamos cuales son las etapas anteriores con el mismo modelo
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer
)

res = classifier("El profesor es bueno todo será sencillo")
print(res)

/Users/lunaflorestorres/.pyenv/versions/3.11.6/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[{'label': 'NEGATIVE', 'score': 0.6551780104637146}]


In [12]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

# ¿Para qué sirve el tokenizer?

Codificación (antes del LLM)

In [13]:
secuencia = "El profesor es bueno todo será sencillo"
res = tokenizer(secuencia)
print(res)

{'input_ids': [101, 3449, 11268, 2229, 2953, 9686, 20934, 16515, 28681, 2080, 26358, 12411, 6895, 7174, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Paso a paso

In [14]:
tokens = tokenizer.tokenize(secuencia) #divide la frase en subdivisiones y se pierde el significado de la clase porque estamos usando un modelo en inglés.
print(tokens)

['el', 'prof', '##es', '##or', 'es', 'bu', '##eno', 'tod', '##o', 'sera', 'sen', '##ci', '##llo']


In [15]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[3449, 11268, 2229, 2953, 9686, 20934, 16515, 28681, 2080, 26358, 12411, 6895, 7174]


Decodificar (después del LLM)

In [16]:
tokenizer.decode(res['input_ids'])

'[CLS] el profesor es bueno todo sera sencillo [SEP]'

# Modelo en español

In [17]:
model_name = "pysentimiento/robertuito-sentiment-analysis"

In [18]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer
)

res = classifier("El profesor es bueno todo será sencillo")
print(res)

/Users/lunaflorestorres/.pyenv/versions/3.11.6/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[{'label': 'POS', 'score': 0.9181905388832092}]


In [19]:
# Codificación del tokenizador

secuencia = "El profesor es bueno todo será sencillo"
res = tokenizer(secuencia)
print(res)

{'input_ids': [0, 459, 5934, 442, 1220, 658, 1504, 9764, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [20]:
tokens = tokenizer.tokenize(secuencia)
print(tokens)

['▁el', '▁profesor', '▁es', '▁bueno', '▁todo', '▁será', '▁sencillo']


In [21]:
token_ids = tokenizer.convert_tokens_to_ids(tokens) #id de cada una de las palabras que luego se convertirán en sus embeding
print(token_ids)

[459, 5934, 442, 1220, 658, 1504, 9764]


In [22]:
# Decodificador

tokenizer.decode(res['input_ids'])

'<s> el profesor es bueno todo será sencillo</s>'

# Guardar modelo y tokenizer en local

In [23]:
model_path = ("./modelo")
tokenizer.save_pretrained(model_path)
model.save_pretrained(model_path)

In [24]:
tokenizer_local = AutoTokenizer.from_pretrained(model_path)
model_local = AutoModelForSequenceClassification.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# También es compatible con Tensorflow (Pytorch)

In [25]:
import torch
import torch.nn.functional as F

In [26]:
sentences = [
    "He estado deseando un curso así toda mi vida",
    "Me encanta HuggingFace"
]

### Tokenizer

In [27]:
batch = tokenizer( # toma las frases y las convierte en números
    sentences,
    padding=True, #  todas las frases tengan el mismo largo
    truncation=True, # no sean demasiado largas
    max_length=512, # no más de 512 palabras
    return_tensors="pt" # queremos los números en un formato que PyTorch
)
print(batch)

{'input_ids': tensor([[    0,   723,  1524, 12667,   471,  4095,   816,  1001,   507,   837,
             2],
        [    0,   474,  2479,   925, 24020, 23912,     2,     1,     1,     1,
             1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}


### Modelo

In [28]:
with torch.no_grad(): # no necesita recordar lo que está haciendo ahora, solo queremos el resultado
    outputs = model(**batch) # Aquí es donde el modelo toma las frases convertidas en números y nos da unos resultados llamados logits.
    predictions = F.softmax(outputs.logits, dim=1) #  Los logits son números que el modelo produce, pero necesitamos convertirlos en probabilidades que sumen 1 (como porcentajes). softmax hace esto por nosotros.
    labels = torch.argmax(predictions, dim=1) # cuál de las probabilidades es la más alta para cada frase. Esto nos dice si la frase es negativa, neutral o positiva.

In [29]:
# Logits
print(outputs) # Son los resultados crudos del modelo antes de convertirlos en probabilidades.


SequenceClassifierOutput(loss=None, logits=tensor([[-0.6811, -0.0606,  0.9667],
        [-1.8555, -0.3444,  2.4575]]), hidden_states=None, attentions=None)


In [30]:
# Neg, Neu, Pos
print(predictions) # Esto nos dice la probabilidad de que cada frase sea negativa, neutral o positiva.

tensor([[0.1241, 0.2309, 0.6450],
        [0.0125, 0.0565, 0.9310]])


In [31]:
# etiquetas
print(labels) # Esto es la categoría final para cada frase (0 para negativa, 1 para neutral, 2 para positiva).

tensor([2, 2])


In [32]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30002, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             